# Predicting Usage of a Video Game Research Server

## Project Final Report

##### Group 13 - Section 005

## Introduction

A research group in the Department of Computer Science at UBC, called The Pacific Laboratory for Artificial Intelligence (PLAI), is working on advancing AI into something more safe and reliable, that we can all trust. One of their projects is a research data collection project that focusses on generative AI (The Pacific Laboratory for Artificial Intelligence, 2023). PLAI has created an online server called "plaicraft.ai", that is a free version fo Minecraft, that anyone can sign up for and play They collect data about the players and how they interact with the server, with the goal of using the data to create AI characters that respond to aspects in the video game in a way that's "smarter" than current non-player characters (Smith, 2023).

Using this collected data, the main goal of our project is to answer the question of "What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?", with a more specefic question of "Can age, gender, experience, and average play time per session predict the subscription of a player?".

#### Description of Datasets

The first dataset we used to answer our question is the `players.csv` dataset.

This dataset is about 196 players in the Minecraft server, including:
- `experience` - (Amateur, Beginner, Regular, Pro, Veteran)
- `subscribe` - Subscription to the newsletter (TRUE/FALSE)
- `hashedEmail` - String of letters and numbers to identify player
- `played_hours`
- `name`
- `gender` - (Agender, Female, Male, Non-Binary, Two-Spirited, Other, Prefer not to say)
- `age`


The second dataset we used to answer our question is the `sessions.csv` dataset.

This dataset is about 1535 sessions of play, including:
- `hashedEmail`
- `start_time` - Including date and time
- `end_time` - Including date and time
- `original_start_time` 
- `original_end_time`

We used, from the `players.csv` data set, the `subscribe` variable as our response variable and `age`, `gender`, `experience` variables as three of our preedictor variables. From both the `players.csv` data set and `sessions.csv`, we used the `hashedEmail` variable to combine the data from both data sets. And from the `sessions.csv` data set, we used the `original_start_time` and `original_end_time` to 

### References

(1) Smith, A. (2023, September 28). plaicraft.ai launch - Pacific Laboratory for Artificial Intelligence. Pacific Laboratory for Artificial Intelligence. https://plai.cs.ubc.ca/2023/09/27/plaicraft/

(2)  The Pacific Laboratory for Artificial Intelligence. (2023, September 28). Home Page - Pacific Laboratory for Artificial Intelligence. Pacific Laboratory for Artificial Intelligence. https://plai.cs.ubc.ca/